In [1]:
import torch

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available, using", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU")

No GPU available, using CPU


In [3]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

train_dir = './train_images'
test_dir = './test_images'

transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2
batch_size = 128

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

In [16]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)


In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

n_epochs = 256
# # for epoch in range(1, n_epochs+1):
# # for data, target in train_loader:
for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

In [9]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images).to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 93 %


## Session 2: 02/10/2023
### 1. Imbalanced Dataset

In [ ]:
import torchsampler import ImbalancedDatasetSampler

train_imbalanced_loader = torch.utils.data.DataLoader(
    train_data,
    sampler=ImbalancedDatasetSampler(train_data),
    batch_size=batch_size,
    **kwargs
)
test_imbalanced_loader = torch.utils.data.DataLoader(
    test_data,
    sampler=ImbalancedDatasetSampler(train_data),
    batch_size=batch_size,
    **kwargs
)

### 2. Data Augmentation 

In [ ]:
# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(256),
        transforms.CenterCrop(36),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))])        
}
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data_augmented = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])
